In this notebook, we test the effect of adding random erasing and color jittering with re-id tasks as they have seemed to perform better in such models (link here: https://www.cv-foundation.org/openaccess/content_cvpr_2014/papers/Li_DeepReID_Deep_Filter_2014_CVPR_paper.pdf, https://kovenyu.com/papers/2019_CVPR_PEDAL.pdf, https://arxiv.org/abs/1708.04896 and https://arxiv.org/abs/1611.05244). Below we test to see if this helps our accuracy.



In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd drive/MyDrive/person-reid/deep-person-reid

/content/drive/MyDrive/person-reid/deep-person-reid


In [3]:
import torchreid
from comp_vis_data_f import CvDataSet

In [4]:
torchreid.data.register_image_dataset('cvision', CvDataSet)

Load data market1501, add transformations of random erasing and color jitter to learn spatial information from different angles from CNN's and remove color learning.


In [ ]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    targets='cvision',
    height=256,    
    width=128,
    combineall=True,
    batch_size_train=64,
    batch_size_test=64,
    transforms=['random_flip', 'random_crop', 'random_erase', 'color_jitter']
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ color jitter
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
+ random erase
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/drive/MyDrive/person-reid/deep-person-reid/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/content/drive/MyDrive/person-reid/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
...6%, 8 MB, 2603 KB/s, 3 seconds passed

This model was built similarly to our prior softmax, cross entropy loss model. It trains the CNN similarly and uses the embedded layers to find the distance between query and gallery.

In [ ]:
model = torchreid.models.build_model(
    name='resnet50',
    num_classes=datamanager.num_train_pids,
    loss='softmax'
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager, model, optimizer, scheduler=scheduler
)


In [ ]:
import os
save_model_path = os.getcwd() + '/log/resnet50-softmax-colorjitter-randomerase-market1501xcv'
save_model_path

'/content/drive/My Drive/person-reid/deep-person-reid/log/resnet50-softmax-colorjitter-randomerase-market1501xcv'

Train the model for 60 epochs, and evaluate on our test set every 10 iterations. We explicitly set the evaluation to calculate using euclidean distance. Lastly, we experimented with freezing the initital layers and training just the classifier for 5 epochs. This is to help the model learn a better local minimum.

In [ ]:
engine.run(save_dir='log/resnet50-softmax-colorjitter-randomerase-market1501xcv', 
           max_epoch=60,
           eval_freq=10,
           print_freq=10,
           test_only=False,
           dist_metric="euclidean",
           fixbase_epoch=5,
           open_layers='classifier'
           )

=> Start training
* Only train classifier (epoch: 1/5)
epoch: [1/60][10/459]	time 0.126 (0.338)	data 0.001 (0.201)	eta 2:35:18	loss 5.8918 (6.0540)	acc 14.0625 (11.4062)	lr 0.000300
epoch: [1/60][20/459]	time 0.123 (0.282)	data 0.000 (0.148)	eta 2:09:16	loss 6.0234 (6.0688)	acc 6.2500 (11.0938)	lr 0.000300
epoch: [1/60][30/459]	time 0.484 (0.292)	data 0.360 (0.161)	eta 2:14:05	loss 6.0891 (6.0681)	acc 7.8125 (10.7292)	lr 0.000300
epoch: [1/60][40/459]	time 0.125 (0.275)	data 0.000 (0.144)	eta 2:05:56	loss 6.0127 (6.0442)	acc 14.0625 (10.5469)	lr 0.000300
epoch: [1/60][50/459]	time 0.373 (0.273)	data 0.248 (0.143)	eta 2:04:53	loss 6.2091 (6.0357)	acc 3.1250 (10.7500)	lr 0.000300
epoch: [1/60][60/459]	time 0.134 (0.263)	data 0.001 (0.133)	eta 2:00:27	loss 5.9124 (6.0359)	acc 9.3750 (10.7552)	lr 0.000300
epoch: [1/60][70/459]	time 0.136 (0.265)	data 0.001 (0.135)	eta 2:01:15	loss 5.8858 (6.0301)	acc 14.0625 (10.9598)	lr 0.000300
epoch: [1/60][80/459]	time 0.131 (0.259)	data 0.000 (0.129)	

The final mAP was 45.6% with Rank-1: at 10.1 and Rank-5 at 25.3%. This was worse than our initial model without the data overload. 

In [ ]:
import os
save_dir = os.getcwd() + '/visrank_cvision_results/visrank_softmax_randomerase_dataset_2'

Evaluation only

In [ ]:
engine.run(save_dir=save_dir, 
           max_epoch=10, 
           test_only=True,
           visrank=True,
           visrank_topk=10,
           dist_metric="euclidean"
           )